In [271]:
import pickle
import requests 
import json
from pprint import pprint
from collections import Counter, defaultdict

from nltk import sent_tokenize, word_tokenize, pos_tag, ne_chunk
#from nameparser.parser import HumanName

In [239]:
with open("guardian_api.txt", 'r') as f:
    api_key = f.next().strip()

In [240]:
def get_content(pg):
    api_url = 'http://content.guardianapis.com/search'
    payload = {
        'api-key':              api_key,
        'page-size':            50,
        'page':                 pg,
        'q':                    'party OR policy OR democrat OR republican', 
        #'order-by':             'newest',
        #'show-editors-picks':   'true',
        #'show-elements':        'image',
        'show-blocks':          'body',
        'show-fields':          'byline, wordcount',
        #'show-tags':            'all',
        #'show-references':      'author',
        'format':               'json'
    }
    response = requests.get(api_url, params=payload)
    #response = response.replace("'", '"')
    #data = json.loads(response)
    data = response.json() # convert json to python-readable format 
    return data


In [243]:
all_data = []
for i in range(1, 6):
    data = get_content(i)
    print data["response"]["status"]
    res = data["response"]["results"]
    all_data += res
#print data["response"]["status"]

ok
ok
ok
ok
ok


In [244]:
print len(all_data)

250


In [245]:
#pprint(res[0])
pprint(all_data[0])

{u'apiUrl': u'http://content.guardianapis.com/us-news/2015/jun/09/carly-fiorina-foreign-policy-republican-interview',
 u'blocks': {u'body': [{u'attributes': {},
                        u'bodyHtml': u'<p>Carly Fiorina has chutzpah. The former CEO of Hewlett Packett isn\u2019t just running for the Republican nomination for president <a href="http://www.theguardian.com/us-news/2015/may/03/carly-fiorina-run-for-president-hewlett-packard">on a platform of managerial expertise</a> after a controversial tenure at that technology company from 1999 to 2005. She\u2019s running as a foreign policy maven despite having never served in elected office.</p> <p>In an interview with the Guardian on Saturday, Fiorina bragged that she \u201cknows more world leaders on the stage\u201d than any of her competitors for the 2016 nomination, a claim she echoed in a speech shortly afterwards at Iowa senator Joni Ernst\u2019s <a href="http://www.theguardian.com/us-news/2015/jun/06/republican-2016-iowa-joni-ernst

In [246]:
print(all_data[0]["blocks"]["body"][0]["bodyTextSummary"])
print(all_data[0]["fields"]["byline"])

Carly Fiorina has chutzpah. The former CEO of Hewlett Packett isn’t just running for the Republican nomination for president on a platform of managerial expertise after a controversial tenure at that technology company from 1999 to 2005. She’s running as a foreign policy maven despite having never served in elected office. In an interview with the Guardian on Saturday, Fiorina bragged that she “knows more world leaders on the stage” than any of her competitors for the 2016 nomination, a claim she echoed in a speech shortly afterwards at Iowa senator Joni Ernst’s inaugural Roast and Ride fundraiser. The former CEO argued that she “conducted business and charitable work” with world leaders “for many years”. She contrasted herself with mere politicians by saying: “I don’t do photo ops. I don’t do codels [congressional delegations].” Fiorina insisted “I know these people. I know how they think about us.” Her foreign policy worldview, though, seemed to be more focused on criticism of the Ob

In [247]:
'''def get_human_names(text):
    tokens = word_tokenize(text)
    pos = pos_tag(tokens)
    sentt = ne_chunk(pos)#, binary = False)
    person_list = []
    person = []
    name = ""
    print sentt
    for subtree in sentt.subtrees(filter=lambda t: t.label() == 'PERSON'):
        for leaf in subtree.leaves():
            person.append(leaf[0])
        if len(person) > 1: #avoid grabbing lone surnames
            for part in person:
                name += part + ' '
            #if name[:-1] not in person_list:
            person_list.append(name[:-1])
            name = ''
        person = []

    return (person_list)'''

def get_human_names(text):
    if " and " in text or " with " in text:
        #don't really deal with multiple authors
        return text
    elif " in " in text:
        #assume same name in different locations is 
        # still the same person
        ind = text.find(" in ")
        return text[:ind]
    else:
        return text

In [281]:
def dd():
    return ""

In [282]:
def make_dict(loc=False):
    authors = []
    names = []
    auth_ar = defaultdict(dd)
    for b in range(len(all_data)):
        if "fields" in all_data[b]:
            if "blocks" in all_data[b]:
                #print b
                nm = str(all_data[b]["fields"]["byline"]).strip()
                #print nm
                authors.append(nm)
                if not loc:
                    nm = get_human_names(str(all_data[b]["fields"]["byline"]))
                names.append(nm)
                auth_ar[nm] += all_data[b]["blocks"]["body"][0]["bodyTextSummary"]
    return names, auth_ar
names, auth_ar = make_dict()
print names, "\n", len(names)
#pprint(authors)
counts = Counter(names)
print(counts)

['Ben Jacobs', 'Frances Perraudin', 'Ben Jacobs', 'Scott Keyes', 'Michael Cohen', 'Ben Jacobs', 'Paul Lewis', 'Ben Jacobs', 'Ben Jacobs in Washington, Sabrina Siddiqui  in Manchester and Jeb Lund', 'Ryan Felton', 'Alan Yuhas with Paul Lewis in Cleveland, Jeb Lund in Tampa, Ben Jacobs in Atlanta, Dan Roberts and Sabrina Siddiqui in Washington and Adam Gabbatt in New York', 'Amanda Holpuch', 'Tom McCarthy', 'Tom Dart', 'Martin Kettle', 'Jeb Lund', 'Sabrina Siddiqui', 'Paul Lewis', 'Observer editorial', 'Dan Roberts', 'Frances Perraudin', 'Paul Lewis', 'Dan Roberts and Sabrina Siddiqui Washington', 'David Boyle', 'Rupert Myers', 'Paul Lewis', 'Anna Leach', 'Ben Jacobs', 'Paul Lewis', 'Jeb Lund', 'Alan Yuhas', 'Scott Lemieux', 'Sabrina Siddiqui', 'Alan Yuhas', 'Paul Lewis', 'Alan Yuhas', 'Patrick Wintour, political editor', 'Frances Perraudin', 'Gary Younge', 'Ben Jacobs in Washington and Alan Yuhas in New York', 'Tom McCarthy', 'Frances Perraudin', 'Sabrina Siddiqui', 'Paul Lewis ', 'Sabr

In [283]:
print len(auth_ar["Frances Perraudin"])

23240


In [284]:
print len(sent_tokenize(auth_ar["Paul Lewis"]))

607


In [285]:
for author, text in auth_ar.iteritems():
    print author, len(sent_tokenize(text))

Mahita Gajanan 22
Frances Perraudin 108
David Boyle 31
Andrew Sparrow, Nadia Khomami and Mark Smith 46
Ben Jacobs in Washington, Sabrina Siddiqui  in Manchester and Jeb Lund 10
John Longworth and Caron Lindsay 30
Paul Lewis and Suzanne Goldenberg Washington 41
Anna Leach 49
Rowena Mason Political correspondent 114
Steven W Thrasher 29
Sabrina Siddiqui 550
Kayla Epstein 53
Patrick Diamond 22
Saeed Kamali Dehghan 25
Gustavo Arellano 30
Tom McCarthy in New York and Ben Jacobs in Baltimore 18
Ana Marie Cox 44
Dan Roberts and Sabrina Siddiqui Washington 20
Edward Helmore 41
Alan Yuhas and Lauren Gambino in New York 17
Denis Campbell and Owen Bowcott 36
Richard Luscombe 26
Nigel M Smith 40
Julian Borger Diplomatic editor 24
Suzanne Goldenberg US environment correspondent, and Sabrina Siddiqui in Derry, New Hampshire 25
Ben Jacobs and Russ Choma in Washington 23
Scott Lemieux 38
Michael Cohen 91
Ben Jacobs in Washington and Tom Dart 31
Rory Carroll 76
Ed Pilkington 98
Rowena Mason, political 

In [292]:
with open("author_articles.pkl", 'w') as out_auth_ar:
    pickle.dump(auth_ar, out_auth_ar)